# GBDT(勾配ブースティング木)

GBDTによる学習は以下のように行われ、それぞれの決定木の分岐および葉のウェイトが定められる。
  - 目的変数と予測値から計算される目的関数を改善するように、決定木を作成してモデルに追加する。
  - 上記の処理をハイパーパラメータで定めた決定木の本数の分だけ繰り返す。

## 必要なライブラリのインポート

In [29]:
import numpy as np
import pandas as pd

# GBDT(勾配ブースティング木)を構成するライブラリ
import xgboost as xgb
import lightgbm as lgb

from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

## データセットの取得

データセットはこちらのURLにアップデートされているものを使用\
https://github.com/ghmagazine/kagglebook/tree/master/input/sample-data

In [30]:
# train_xは学習データ、train_yは目的変数、test_xはテストデータ
train = pd.read_csv('train_preprocessed.csv')
test = pd.read_csv('test_preprocessed.csv')

In [31]:
train.head()

,age,sex,height,weight,product,amount,medical_info_a1,medical_info_a2,medical_info_a3,medical_info_b1,...,medical_keyword_6,medical_keyword_7,medical_keyword_8,medical_keyword_9,medical_keyword_10,year,month,day,yearmonth,target
0,50,1,166.445608,65.016732,9,7000000,134,202,1,11,...,1,0,1,0,0,2015,2,3,24182,0
1,68,0,164.334615,56.544217,0,7000000,438,263,3,14,...,0,1,1,0,0,2015,5,9,24185,0
2,77,1,167.462917,54.242267,2,6000000,313,325,1,18,...,1,0,1,0,0,2016,2,13,24194,1
3,17,1,177.097725,71.147762,3,8000000,342,213,2,11,...,0,0,1,0,0,2015,7,6,24187,0
4,62,0,158.165788,65.240697,1,9000000,327,102,0,14,...,0,1,1,1,0,2016,9,17,24201,1


In [32]:
train.shape

(10000, 29)

In [33]:
train.drop(['target'], axis=1)

,age,sex,height,weight,product,amount,medical_info_a1,medical_info_a2,medical_info_a3,medical_info_b1,...,medical_keyword_5,medical_keyword_6,medical_keyword_7,medical_keyword_8,medical_keyword_9,medical_keyword_10,year,month,day,yearmonth
0,50,1,166.445608,65.016732,9,7000000,134,202,1,11,...,0,1,0,1,0,0,2015,2,3,24182
1,68,0,164.334615,56.544217,0,7000000,438,263,3,14,...,0,0,1,1,0,0,2015,5,9,24185
2,77,1,167.462917,54.242267,2,6000000,313,325,1,18,...,0,1,0,1,0,0,2016,2,13,24194
3,17,1,177.097725,71.147762,3,8000000,342,213,2,11,...,0,0,0,1,0,0,2015,7,6,24187
4,62,0,158.165788,65.240697,1,9000000,327,102,0,14,...,0,0,1,1,1,0,2016,9,17,24201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,61,1,182.729800,73.393777,1,2000000,189,232,7,17,...,0,0,0,1,1,0,2015,10,21,24190
9996,33,0,167.701136,75.006529,8,9000,426,202,3,19,...,0,0,0,1,1,0,2015,5,28,24185
9997,44,0,145.609998,47.739397,8,1000,370,274,1,11,...,0,0,0,1,0,1,2016,2,29,24194
9998,34,0,165.796017,57.567695,6,5000,291,105,1,13,...,0,1,1,1,1,0,2016,2,27,24194


In [34]:
train.head().iloc[:, :28]

,age,sex,height,weight,product,amount,medical_info_a1,medical_info_a2,medical_info_a3,medical_info_b1,...,medical_keyword_5,medical_keyword_6,medical_keyword_7,medical_keyword_8,medical_keyword_9,medical_keyword_10,year,month,day,yearmonth
0,50,1,166.445608,65.016732,9,7000000,134,202,1,11,...,0,1,0,1,0,0,2015,2,3,24182
1,68,0,164.334615,56.544217,0,7000000,438,263,3,14,...,0,0,1,1,0,0,2015,5,9,24185
2,77,1,167.462917,54.242267,2,6000000,313,325,1,18,...,0,1,0,1,0,0,2016,2,13,24194
3,17,1,177.097725,71.147762,3,8000000,342,213,2,11,...,0,0,0,1,0,0,2015,7,6,24187
4,62,0,158.165788,65.240697,1,9000000,327,102,0,14,...,0,0,1,1,1,0,2016,9,17,24201


In [35]:
test.head()

,age,sex,height,weight,product,amount,medical_info_a1,medical_info_a2,medical_info_a3,medical_info_b1,...,medical_keyword_5,medical_keyword_6,medical_keyword_7,medical_keyword_8,medical_keyword_9,medical_keyword_10,year,month,day,yearmonth
0,49,1,187.431987,81.008363,1,1000000,302,212,1,10,...,0,1,0,1,0,0,2016,12,6,24204
1,79,1,171.632630,71.067812,6,2000,197,469,0,14,...,0,0,0,0,1,1,2016,9,3,24201
2,78,0,163.543983,64.032098,0,4000000,247,225,2,17,...,0,1,0,1,0,0,2015,4,10,24184
3,26,1,150.391858,52.322910,2,1000000,108,228,0,15,...,0,0,1,0,0,0,2016,4,17,24196
4,14,1,165.835167,67.008154,2,4000000,181,90,2,11,...,0,0,0,1,0,0,2015,1,26,24181


In [36]:
list_idx = [0, 1, 2]
train.iloc[list_idx]

,age,sex,height,weight,product,amount,medical_info_a1,medical_info_a2,medical_info_a3,medical_info_b1,...,medical_keyword_6,medical_keyword_7,medical_keyword_8,medical_keyword_9,medical_keyword_10,year,month,day,yearmonth,target
0,50,1,166.445608,65.016732,9,7000000,134,202,1,11,...,1,0,1,0,0,2015,2,3,24182,0
1,68,0,164.334615,56.544217,0,7000000,438,263,3,14,...,0,1,1,0,0,2015,5,9,24185,0
2,77,1,167.462917,54.242267,2,6000000,313,325,1,18,...,1,0,1,0,0,2016,2,13,24194,1


In [37]:
train_x = train.iloc[:, :28]
train_y = train['target']
test_x = test.copy()

In [38]:
# KFoldクロスバリデーションによる分割の1つを使用し、学習データとバリデーションデータに分ける
kf = KFold(n_splits=4, shuffle=True, random_state=71)
##  トレーニングデータセットとテストデータセットにそれぞれ振り分けた要素番号のリスト
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

In [39]:
list(kf.split(train_x))[0]

(array([   0,    1,    2, ..., 9996, 9998, 9999]),
 array([   6,   10,   11, ..., 9994, 9995, 9997]))

## xgboostライブラリを使用

In [40]:
# 特徴量と目的変数をxgboostのデータ後続に変換する
dtrain = xgb.DMatrix(tr_x, label=tr_y)
dvalid = xgb.DMatrix(va_x, label=va_y)
dtest = xgb.DMatrix(test_x)

In [41]:
# ハイパーパラメータの設定
params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
num_round = 50

In [44]:
# 学習の実行
# バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
# watchlistには学習データおよびバリデーションデータをセットする
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
model = xgb.train(params, dtrain, num_round, evals=watchlist)

[0]	train-error:0.128533	eval-error:0.1516
[1]	train-error:0.115333	eval-error:0.146
[2]	train-error:0.109333	eval-error:0.1376
[3]	train-error:0.105333	eval-error:0.1364
[4]	train-error:0.096933	eval-error:0.1384
[5]	train-error:0.094667	eval-error:0.1364
[6]	train-error:0.087333	eval-error:0.1296
[7]	train-error:0.084933	eval-error:0.1244
[8]	train-error:0.078133	eval-error:0.1208
[9]	train-error:0.073733	eval-error:0.1172
[10]	train-error:0.068667	eval-error:0.116
[11]	train-error:0.064933	eval-error:0.1164
[12]	train-error:0.062267	eval-error:0.1112
[13]	train-error:0.060533	eval-error:0.1116
[14]	train-error:0.0568	eval-error:0.1112
[15]	train-error:0.0504	eval-error:0.1048
[16]	train-error:0.0492	eval-error:0.1004
[17]	train-error:0.0464	eval-error:0.1016
[18]	train-error:0.044267	eval-error:0.1028
[19]	train-error:0.043467	eval-error:0.1012
[20]	train-error:0.038667	eval-error:0.1016
[21]	train-error:0.036533	eval-error:0.1004
[22]	train-error:0.034933	eval-error:0.0996
[23]	tra

In [45]:
# バリデーションデータでのスコアの確認
va_pred = model.predict(dvalid)
score = log_loss(va_y, va_pred)
print(f'logloss: {score:.4f}')

logloss: 0.2223


In [46]:
# 予測（二値の予測値ではなく、1である確率を出力するようにしている）
pred = model.predict(dtest)

In [47]:
pred

array([0.2064009 , 0.02400707, 0.00388634, ..., 0.83468455, 0.00239013,
       0.22417206], dtype=float32)

### 二つのGBDTライブラリの精度があまり変わらない場合は速度が早い方のライブラリを使用する